In [1]:
###Neural Network

# import sys
# !conda install --yes --prefix {sys.prefix} keras

In [2]:
import pandas as pd
import numpy as np
from pandas import datetime
from sklearn import preprocessing

import h5py

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
%matplotlib inline

In [3]:

# %% Setup path to data files

path = "/Users/tejasvibelsare/Documents/DM term project/"

filename_train = "train.csv"
filename_test = "test.csv"
filename_feature = "features.csv"
filename_store = "stores.csv"

In [4]:
# %% Load data

df_store = pd.read_csv(path + filename_store)
df_feature = pd.read_csv(path + filename_feature)
df_train = pd.read_csv(path + filename_train)
df_test = pd.read_csv(path + filename_test)

In [5]:
#### Joining tables

combined_train = pd.merge(df_train, df_store, how="left", on="Store")
combined_test = pd.merge(df_test, df_store, how="left", on="Store")

print(combined_train.head(),"\n", combined_train.shape,"\n")
print(combined_test.head(),"\n", combined_test.shape,"\n")

   Store  Dept        Date  Weekly_Sales  IsHoliday Type    Size
0      1     1  2010-02-05      24924.50      False    A  151315
1      1     1  2010-02-12      46039.49       True    A  151315
2      1     1  2010-02-19      41595.55      False    A  151315
3      1     1  2010-02-26      19403.54      False    A  151315
4      1     1  2010-03-05      21827.90      False    A  151315 
 (421570, 7) 

   Store  Dept        Date  IsHoliday Type    Size
0      1     1  2012-11-02      False    A  151315
1      1     1  2012-11-09      False    A  151315
2      1     1  2012-11-16      False    A  151315
3      1     1  2012-11-23       True    A  151315
4      1     1  2012-11-30      False    A  151315 
 (115064, 6) 



In [6]:
combined_train = pd.merge(combined_train, df_feature, how = "inner", on=["Store","Date"])
combined_test = pd.merge(combined_test, df_feature, how = "inner", on=["Store","Date"])

print(combined_train.head())
print(combined_test.head())

   Store  Dept        Date  Weekly_Sales  IsHoliday_x Type    Size  \
0      1     1  2010-02-05      24924.50        False    A  151315   
1      1     2  2010-02-05      50605.27        False    A  151315   
2      1     3  2010-02-05      13740.12        False    A  151315   
3      1     4  2010-02-05      39954.04        False    A  151315   
4      1     5  2010-02-05      32229.38        False    A  151315   

   Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0        42.31       2.572        NaN        NaN        NaN        NaN   
1        42.31       2.572        NaN        NaN        NaN        NaN   
2        42.31       2.572        NaN        NaN        NaN        NaN   
3        42.31       2.572        NaN        NaN        NaN        NaN   
4        42.31       2.572        NaN        NaN        NaN        NaN   

   MarkDown5         CPI  Unemployment  IsHoliday_y  
0        NaN  211.096358         8.106        False  
1        NaN  211.096358  

In [7]:
# Drop Extra IsHoliday_y from both combined data set
combined_train = combined_train.drop(["IsHoliday_y"], axis=1)
combined_test = combined_test.drop(["IsHoliday_y"], axis=1)

print(combined_train.head())
print(combined_test.head())

   Store  Dept        Date  Weekly_Sales  IsHoliday_x Type    Size  \
0      1     1  2010-02-05      24924.50        False    A  151315   
1      1     2  2010-02-05      50605.27        False    A  151315   
2      1     3  2010-02-05      13740.12        False    A  151315   
3      1     4  2010-02-05      39954.04        False    A  151315   
4      1     5  2010-02-05      32229.38        False    A  151315   

   Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0        42.31       2.572        NaN        NaN        NaN        NaN   
1        42.31       2.572        NaN        NaN        NaN        NaN   
2        42.31       2.572        NaN        NaN        NaN        NaN   
3        42.31       2.572        NaN        NaN        NaN        NaN   
4        42.31       2.572        NaN        NaN        NaN        NaN   

   MarkDown5         CPI  Unemployment  
0        NaN  211.096358         8.106  
1        NaN  211.096358         8.106  
2        Na

In [8]:
combined_train.describe()
##### Weekly_Sales, Markdown2 & Markdown3 have negative values in the combined_train set, which need to be replaced by 0

,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,150681.000000,111248.000000,137091.000000,134967.000000,151432.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123,136727.915739,60.090059,3.361027,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,171.201947,7.960289
std,12.785297,30.492054,22711.183519,60980.583328,18.447931,0.458515,8291.221345,9475.357325,9623.078290,6292.384031,5962.887455,39.159276,1.863296
min,1.000000,1.000000,-4988.940000,34875.000000,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,126.064000,3.879000
25%,11.000000,18.000000,2079.650000,93638.000000,46.680000,2.933000,2240.270000,41.600000,5.080000,504.220000,1878.440000,132.022667,6.891000
50%,22.000000,37.000000,7612.030000,140167.000000,62.090000,3.452000,5347.450000,192.000000,24.600000,1481.310000,3359.450000,182.318780,7.866000
75%,33.000000,74.000000,20205.852500,202505.000000,74.280000,3.738000,9210.900000,1926.940000,103.990000,3595.040000,5563.800000,212.416993,8.572000
max,45.000000,99.000000,693099.360000,219622.000000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000


In [9]:
combined_test.describe()
#NOTE: Markdown1, Markdown2, Markdown3 & Markdown5 have negative values in the combined_test set, which need to be replaced by 0

,Store,Dept,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,114915.000000,86437.000000,105235.000000,102176.000000,115064.000000,76902.000000,76902.000000
mean,22.238207,44.339524,136497.688921,53.941804,3.581546,7689.216439,3734.051729,2403.088666,3356.219071,3922.681189,176.961347,6.868733
std,12.809930,30.656410,61106.926438,18.724153,0.239442,10698.760716,8323.495014,13767.939313,7570.501545,19445.150745,41.239967,1.583427
min,1.000000,1.000000,34875.000000,-7.290000,2.872000,-2781.450000,-35.740000,-179.260000,0.220000,-185.170000,131.236226,3.684000
25%,11.000000,18.000000,93638.000000,39.820000,3.431000,1966.460000,180.350000,15.100000,155.460000,1309.300000,138.402033,5.771000
50%,22.000000,37.000000,140167.000000,54.470000,3.606000,4842.290000,742.590000,78.260000,840.940000,2390.430000,192.304445,6.806000
75%,33.000000,74.000000,202505.000000,67.350000,3.766000,9439.140000,2735.670000,272.580000,3096.920000,4227.270000,223.244532,8.036000
max,45.000000,99.000000,219622.000000,101.950000,4.125000,103184.980000,71074.170000,149483.310000,65344.640000,771448.100000,228.976456,10.199000


In [ ]:
# Check for count of missing values in combined_train, combined_test

print(combined_test.isnull().sum())
print(combined_train.isnull().sum())

Store               0
Dept                0
Date                0
IsHoliday_x         0
Type                0
Size                0
Temperature         0
Fuel_Price          0
MarkDown1         149
MarkDown2       28627
MarkDown3        9829
MarkDown4       12888
MarkDown5           0
CPI             38162
Unemployment    38162
dtype: int64
Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Type                 0
Size                 0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
dtype: int64


In [ ]:
processed_train = combined_train.fillna(0)
processed_test = combined_test.fillna(0)

In [ ]:
##Replace -ve Markdown values by 0 in both processed_train and processed_test
# %% Processed_train
processed_train.loc[processed_train['Weekly_Sales'] < 0.0,'Weekly_Sales'] = 0.0
processed_train.loc[processed_train['MarkDown2'] < 0.0,'MarkDown2'] = 0.0
processed_train.loc[processed_train['MarkDown3'] < 0.0,'MarkDown3'] = 0.0
processed_train.describe()

,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.467250,136727.915739,60.090059,3.361027,2590.074819,880.070274,468.092929,1083.132268,1662.772385,171.201947,7.960289
std,12.785297,30.492054,22711.032446,60980.583328,18.447931,0.458515,6052.385934,5084.520381,5528.872994,3894.529945,4207.629321,39.159276,1.863296
min,1.000000,1.000000,0.000000,34875.000000,-2.060000,2.472000,0.000000,0.000000,0.000000,0.000000,0.000000,126.064000,3.879000
25%,11.000000,18.000000,2079.650000,93638.000000,46.680000,2.933000,0.000000,0.000000,0.000000,0.000000,0.000000,132.022667,6.891000
50%,22.000000,37.000000,7612.030000,140167.000000,62.090000,3.452000,0.000000,0.000000,0.000000,0.000000,0.000000,182.318780,7.866000
75%,33.000000,74.000000,20205.852500,202505.000000,74.280000,3.738000,2809.050000,2.200000,4.540000,425.290000,2168.040000,212.416993,8.572000
max,45.000000,99.000000,693099.360000,219622.000000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000


In [ ]:
# Processed_test
processed_test.loc[processed_test['MarkDown1'] < 0.0,'MarkDown1'] = 0.0
processed_test.loc[processed_test['MarkDown2'] < 0.0,'MarkDown2'] = 0.0
processed_test.loc[processed_test['MarkDown3'] < 0.0,'MarkDown3'] = 0.0
processed_test.loc[processed_test['MarkDown5'] < 0.0,'MarkDown5'] = 0.0
processed_test.describe()

,Store,Dept,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000
mean,22.238207,44.339524,136497.688921,53.941804,3.581546,7681.106189,2805.085103,2198.021241,2980.298267,3922.806060,118.270541,4.590657
std,12.809930,30.656410,61106.926438,18.724153,0.239442,10693.903634,7392.542823,13183.852299,7212.030921,19445.125049,89.878361,3.483338
min,1.000000,1.000000,34875.000000,-7.290000,2.872000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,18.000000,93638.000000,39.820000,3.431000,1963.550000,0.000000,7.770000,70.370000,1309.300000,0.000000,0.000000
50%,22.000000,37.000000,140167.000000,54.470000,3.606000,4828.720000,316.880000,60.000000,600.580000,2390.430000,138.402033,5.771000
75%,33.000000,74.000000,202505.000000,67.350000,3.766000,9427.410000,1575.850000,244.200000,2627.850000,4227.270000,201.212230,7.293000
max,45.000000,99.000000,219622.000000,101.950000,4.125000,103184.980000,71074.170000,149483.310000,65344.640000,771448.100000,228.976456,10.199000


In [ ]:
print(processed_train.dtypes, processed_test.dtypes)
cat_col = ['IsHoliday_x','Type']

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday_x        bool
Type             object
Size              int64
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
dtype: object Store             int64
Dept              int64
Date             object
IsHoliday_x        bool
Type             object
Size              int64
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
dtype: object


In [ ]:
from sklearn import preprocessing

for col in cat_col:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(processed_train[col].values.astype('str'))
    processed_train[col] = lbl.transform(processed_train[col].values.astype('str'))

In [ ]:
for col in cat_col:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(processed_test[col].values.astype('str'))
    processed_test[col] = lbl.transform(processed_test[col].values.astype('str'))

In [ ]:
processed_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106
1,1,2,2010-02-05,50605.27,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106
2,1,3,2010-02-05,13740.12,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106
3,1,4,2010-02-05,39954.04,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106
4,1,5,2010-02-05,32229.38,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106


In [ ]:
# Rearrange the response columns ('Weekly_Sales')

processed_train = processed_train[['Store', 'Dept', 'Date', 'Unemployment', 'IsHoliday_x', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Weekly_Sales']]

In [ ]:
import tensorflow.keras
from sklearn.preprocessing import MinMaxScaler

In [ ]:
print(processed_train['Date'].min(), processed_train['Date'].max())

2010-02-05 2012-10-26


In [ ]:
from datetime import datetime

split_date = pd.datetime(2012,8,24)
processed_train['Date'] = pd.to_datetime(processed_train['Date'])
train_set = processed_train.loc[processed_train['Date'] <= split_date]
dev_set = processed_train.loc[processed_train['Date'] > split_date]

In [ ]:
split_date_dev = pd.datetime(2012,9,25)
val_set = dev_set.loc[dev_set['Date'] <= split_date_dev]
test_set = dev_set.loc[dev_set['Date'] > split_date_dev]

In [ ]:
train_set = train_set.set_index('Date')
val_set = val_set.set_index('Date')
test_set = test_set.set_index('Date')

In [ ]:
train_set_array = train_set.iloc[:,:].values
val_set_array = val_set.iloc[:,:].values
test_set_array = test_set.iloc[:,:].values

print("Shape of train, val and test array:\n",train_set_array.shape,"\n",val_set_array.shape,"\n",test_set_array.shape)

Shape of train, val and test array:
 (394879, 15) 
 (11854, 15) 
 (14837, 15)


In [ ]:
# Scaling
sc = MinMaxScaler(feature_range=(0,1))
train_set_scaled = sc.fit_transform(train_set_array[:,:])
val_set_scaled = sc.fit_transform(val_set_array[:,:])
test_set_scaled = sc.fit_transform(test_set_array[:,:])

print(train_set_scaled.shape, val_set_scaled.shape, test_set_scaled.shape)

(394879, 15) (11854, 15) (14837, 15)


In [ ]:
X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

X_train, y_train = train_set_scaled[:,:-1], train_set_scaled[:,-1]
X_val, y_val = val_set_scaled[:,:-1], val_set_scaled[:,-1]
X_test, y_test = test_set_scaled[:,:-1], test_set_scaled[:,-1]

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(394879, 14) (394879,) (11854, 14) (11854,) (14837, 14) (14837,)


In [ ]:
# Create data structure

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)

(394879, 1, 14) (11854, 1, 14) (14837, 1, 14)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

In [ ]:
# Initialising RNN
regressor = Sequential()

In [ ]:
# Adding the first LSTM layer and some Dropout regularization
# Dropout regularization is added to avoid overfitting

regressor.add(LSTM(units = 10, return_sequences = True, activation = 'relu', input_shape = (X_train.shape[1], 14)))
regressor.add(Dropout(0.5))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# adding a second LSTM layer and some dropout regularization
regressor.add(LSTM(units = 10, return_sequences = True, activation = 'relu'))
regressor.add(Dropout(0.5))

In [ ]:
# # adding a third LSTM layer and some dropout regularization
# regressor.add(LSTM(units = 10, return_sequences = True, activation = 'relu'))
# regressor.add(Dropout(0.7))

In [ ]:
# adding a fourth LSTM layer and some dropout regularization
regressor.add(LSTM(units = 10, return_sequences = False, activation = 'relu'))
regressor.add(Dropout(0.5))

In [ ]:
# Adding the output layer
#regressor.add(Flatten())
regressor.add(Dense(units=1, activation = 'sigmoid'))

In [ ]:
# Compiling the RNN
regressor.compile(optimizer='adam', 
                  loss='mean_squared_error', 
                  metrics=['accuracy'])

Instructions for updating:
Use tf.cast instead.


In [ ]:
# Fitting the RNN to the training set
history = regressor.fit(X_train, 
              y_train, 
              epochs = 20, 
              batch_size = 512, 
              validation_data = (X_val, y_val),
              verbose = 1)

Train on 394879 samples, validate on 11854 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
